## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

May. 4th May. 11th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,May-9-2025,Condo/Co-op,10662 Ocean Palm Way #101,Boynton Beach,FL,33437.0,269000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/10662-...,Beaches MLS,RX-11069132,N,Y,26.518344,-80.128832


In [8]:
print('Input county name:')
county = input()

Input county name:
Palm Beach


In [9]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-05-09    27
2025-05-05    22
2025-05-08    16
2025-05-07    16
2025-05-06    16
Name: count, dtype: int64

In [13]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [14]:
df_top_ten = df2.head(10)

In [15]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
97,PAST SALE,2025-05-06,Condo/Co-op,2 N Breakers Row Unit S-T1,Palm Beach,FL,33480.0,6000000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10879239,N,Y,26.717140,-80.034673
54,PAST SALE,2025-05-09,Condo/Co-op,425 Beach Rd Ph -O,Tequesta,FL,33469.0,4100000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/425-Beach-Rd...,Beaches MLS,RX-11056254,N,Y,26.955362,-80.077420
64,PAST SALE,2025-05-07,Condo/Co-op,2700 Donald Ross Rd #412,Palm Beach Gardens,FL,33410.0,3900000.0,4.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11049233,N,Y,26.880648,-80.071850
6,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #215,Boca Raton,FL,33432.0,3211000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11089230,N,Y,26.347042,-80.083200
3,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #217,Boca Raton,FL,33432.0,2900000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11089244,N,Y,26.347042,-80.083200
30,PAST SALE,2025-05-06,Condo/Co-op,44 Cocoanut Row Unit 321a,Palm Beach,FL,33480.0,2500000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Beaches MLS,RX-11071642,N,Y,26.715065,-80.042094
51,PAST SALE,2025-05-06,Condo/Co-op,604 Captains Way,Jupiter,FL,33477.0,2350000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/604-Captains...,Beaches MLS,RX-11049155,N,Y,26.909631,-80.085249
60,PAST SALE,2025-05-05,Condo/Co-op,13633 Treasure Cove Cir,North Palm Beach,FL,33408.0,2200000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/136...,Beaches MLS,RX-11048862,N,Y,26.876802,-80.065047
61,PAST SALE,2025-05-05,Condo/Co-op,13631 Treasure Cove Cir,North Palm Beach,FL,33408.0,2200000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/136...,Beaches MLS,RX-11048869,N,Y,26.876802,-80.065047
62,PAST SALE,2025-05-05,Condo/Co-op,13629 Treasure Cove Cir,North Palm Beach,FL,33408.0,2200000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/136...,Beaches MLS,RX-11048850,N,Y,26.876802,-80.065047


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [16]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [17]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [18]:
response_df = pd.DataFrame(response_list)

In [19]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [20]:
df_top_ten = merged_df

In [21]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-05-06,Condo/Co-op,2 N Breakers Row Unit S-T1,Palm Beach,FL,33480.0,6000000.0,3.0,3.5,...,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Laura Semler,Brown Harris Stevens of PB,None,None,Christian Angle,Christian Angle Real Estate,None,None
1,PAST SALE,2025-05-09,Condo/Co-op,425 Beach Rd Ph -O,Tequesta,FL,33469.0,4100000.0,3.0,3.5,...,-80.077420,https://www.redfin.com/FL/Jupiter/425-Beach-Rd...,John Holland,Holland Group Real Estate LLC,Emily Sullivan,Holland Group Real Estate LLC,Traci Chivers,Premier Properties of South Fl,None,None
2,PAST SALE,2025-05-07,Condo/Co-op,2700 Donald Ross Rd #412,Palm Beach Gardens,FL,33410.0,3900000.0,4.0,3.5,...,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Ken Meierling,Engel & Voelkers Jupiter RE,None,None
3,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #215,Boca Raton,FL,33432.0,3211000.0,2.0,3.0,...,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Pamela Levine,Luxury Partners Realty,Jeff Levine,Luxury Partners Realty
4,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #217,Boca Raton,FL,33432.0,2900000.0,2.0,2.5,...,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Rochelle LeCavalier,Douglas Elliman,None,None
5,PAST SALE,2025-05-06,Condo/Co-op,44 Cocoanut Row Unit 321a,Palm Beach,FL,33480.0,2500000.0,2.0,2.0,...,-80.042094,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Mark Whitehead,Coffee Cake And Real Estate,Brie Segota,Coffee Cake And Real Estate,Non Agent,Non-Member Selling Office,None,None
6,PAST SALE,2025-05-06,Condo/Co-op,604 Captains Way,Jupiter,FL,33477.0,2350000.0,3.0,2.0,...,-80.085249,https://www.redfin.com/FL/Jupiter/604-Captains...,Stephen Rockoff,The Corcoran Group,None,None,Crystal Condit,Waterfront Properties & Club C,None,None
7,PAST SALE,2025-05-05,Condo/Co-op,13633 Treasure Cove Cir,North Palm Beach,FL,33408.0,2200000.0,3.0,2.0,...,-80.065047,https://www.redfin.com/FL/North-Palm-Beach/136...,James Edwards,"Sotheby's Intl. Realty, Inc.",Patricia Mahaney,"Sotheby's Intl. Realty, Inc.",Patrick Kelly,Coastal Realty Group Sales & M,None,None
8,PAST SALE,2025-05-05,Condo/Co-op,13631 Treasure Cove Cir,North Palm Beach,FL,33408.0,2200000.0,3.0,2.0,...,-80.065047,https://www.redfin.com/FL/North-Palm-Beach/136...,James Edwards,"Sotheby's Intl. Realty, Inc.",Patricia Mahaney,"Sotheby's Intl. Realty, Inc.",Patrick Kelly,Coastal Realty Group Sales & M,None,None
9,PAST SALE,2025-05-05,Condo/Co-op,13629 Treasure Cove Cir,North Palm Beach,FL,33408.0,2200000.0,3.0,2.0,...,-80.065047,https://www.redfin.com/FL/North-Palm-Beach/136...,James Edwards,"Sotheby's Intl. Realty, Inc.",Patricia Mahaney,"Sotheby's Intl. Realty, Inc.",Patrick Kelly,Coastal Realty Group Sales & M,None,None


## Current Week's Values

In [22]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

97


In [23]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$717,106


In [24]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$391


In [25]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$69,559,250


In [26]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           97              717106          391                     69559250.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [27]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
194
-----------
Input Previous Week Condo Average Sales Price:
494991
-----------
Input Previous Week Condo Average PSF:
333
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
96028333.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [28]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [29]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [30]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [31]:
pd.set_option('display.max_columns',None)

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-06,Condo/Co-op,2 N Breakers Row Unit S-T1,Palm Beach,FL,33480.0,6000000.0,3.0,3.5,Two North Breakers Row Condo,3243.0,NaN,1986.0,NaN,1850.0,13078.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10879239,N,Y,26.71714,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Laura Semler,Brown Harris Stevens of PB,None,None,Christian Angle,Christian Angle Real Estate,None,None,1,orange


In [33]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [34]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-06,Condo/Co-op,2 N Breakers Row Unit S-T1,Palm Beach,FL,33480.0,6000000.0,3.0,3.5,Two North Breakers Row Condo,3243.0,NaN,1986.0,NaN,1850.0,13078.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10879239,N,Y,26.717140,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Laura Semler,Brown Harris Stevens of PB,None,None,Christian Angle,Christian Angle Real Estate,None,None,1,orange
1,PAST SALE,2025-05-09,Condo/Co-op,425 Beach Rd Ph -O,Tequesta,FL,33469.0,4100000.0,3.0,3.5,Cliveden Jupiter Island,2750.0,NaN,1996.0,NaN,1491.0,2378.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/425-Beach-Rd...,Beaches MLS,RX-11056254,N,Y,26.955362,-80.077420,https://www.redfin.com/FL/Jupiter/425-Beach-Rd...,John Holland,Holland Group Real Estate LLC,Emily Sullivan,Holland Group Real Estate LLC,Traci Chivers,Premier Properties of South Fl,None,None,2,blue
2,PAST SALE,2025-05-07,Condo/Co-op,2700 Donald Ross Rd #412,Palm Beach Gardens,FL,33410.0,3900000.0,4.0,3.5,Azure,3466.0,NaN,2016.0,NaN,1125.0,3635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11049233,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Ken Meierling,Engel & Voelkers Jupiter RE,None,None,3,blue
3,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #215,Boca Raton,FL,33432.0,3211000.0,2.0,3.0,Alina Residences,2683.0,180.0,2024.0,NaN,1197.0,2291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11089230,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Pamela Levine,Luxury Partners Realty,Jeff Levine,Luxury Partners Realty,4,blue
4,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #217,Boca Raton,FL,33432.0,2900000.0,2.0,2.5,Alina Residences,2429.0,180.0,2024.0,NaN,1194.0,2050.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11089244,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Rochelle LeCavalier,Douglas Elliman,None,None,5,blue
5,PAST SALE,2025-05-06,Condo/Co-op,44 Cocoanut Row Unit 321a,Palm Beach,FL,33480.0,2500000.0,2.0,2.0,Palm Beach Towers,1621.0,NaN,1956.0,NaN,1542.0,2193.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Beaches MLS,RX-11071642,N,Y,26.715065,-80.042094,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Mark Whitehead,Coffee Cake And Real Estate,Brie Segota,Coffee Cake And Real Estate,Non Agent,Non-Member Selling Office,None,None,6,blue
6,PAST SALE,2025-05-06,Condo/Co-op,604 Captains Way,Jupiter,FL,33477.0,2350000.0,3.0,2.0,Admirals Cove,2160.0,NaN,1988.0,NaN,1088.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/604-Captains...,Beaches MLS,RX-11049155,N,Y,26.909631,-80.085249,https://www.redfin.com/FL/Jupiter/604-Captains...,Stephen Rockoff,The Corcoran Group,None,None,Crystal Condit,Waterfront Properties & Club C,None,None,7,blue
7,PAST SALE,2025-05-05,Condo/Co-op,13633 Treasure Cove Cir,North Palm Beach,FL,33408.0,2200000.0,3.0,2.0,Carriage Homes AT Frenchmans Harbor Condo,2493.0,NaN,2012.0,NaN,882.0,2397.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/136...,Beaches MLS,RX-11048862,N,Y,26.876802,-80.065047,https://www.redfin.com/FL/North-Palm-Beach/136...,James Edwards,"Sotheby's Intl. Realty, Inc.",Patricia Mahaney,"Sotheby's Intl. Re

In [35]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [36]:
m.save('index.html')

## Data snagger

In [37]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [38]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-06,Condo/Co-op,2 N Breakers Row Unit S-T1,Palm Beach,FL,33480.0,6000000.0,3.0,3.5,Two North Breakers Row Condo,3243.0,NaN,1986.0,NaN,1850.0,13078.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10879239,N,Y,26.71714,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Laura Semler,Brown Harris Stevens of PB,None,None,Christian Angle,Christian Angle Real Estate,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [39]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_05122025


# CREATE TEMPLATE 

In [40]:
muni_set = set(df_top_ten['CITY'])

In [41]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [42]:
df_top_ten.reset_index(inplace=True,drop=True)

In [43]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [44]:
top_sale

'Two North Breakers Row Condo at 2 N Breakers Row Unit S-T1 in Palm Beach'

In [45]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [46]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-06,Condo/Co-op,2 N Breakers Row Unit S-T1,Palm Beach,FL,33480.0,6000000.0,3.0,3.5,Two North Breakers Row Condo,3243.0,NaN,1986.0,NaN,1850.0,13078.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10879239,N,Y,26.717140,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Laura Semler,Brown Harris Stevens of PB,None,None,Christian Angle,Christian Angle Real Estate,None,None,1,orange,Two North Breakers Row Condo at 2 N Breakers R...
1,PAST SALE,2025-05-09,Condo/Co-op,425 Beach Rd Ph -O,Tequesta,FL,33469.0,4100000.0,3.0,3.5,Cliveden Jupiter Island,2750.0,NaN,1996.0,NaN,1491.0,2378.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/425-Beach-Rd...,Beaches MLS,RX-11056254,N,Y,26.955362,-80.077420,https://www.redfin.com/FL/Jupiter/425-Beach-Rd...,John Holland,Holland Group Real Estate LLC,Emily Sullivan,Holland Group Real Estate LLC,Traci Chivers,Premier Properties of South Fl,None,None,2,blue,Cliveden Jupiter Island at 425 Beach Rd Ph -O ...
2,PAST SALE,2025-05-07,Condo/Co-op,2700 Donald Ross Rd #412,Palm Beach Gardens,FL,33410.0,3900000.0,4.0,3.5,Azure,3466.0,NaN,2016.0,NaN,1125.0,3635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11049233,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Ken Meierling,Engel & Voelkers Jupiter RE,None,None,3,blue,Azure at 2700 Donald Ross Rd #412 in Palm Beac...
3,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #215,Boca Raton,FL,33432.0,3211000.0,2.0,3.0,Alina Residences,2683.0,180.0,2024.0,NaN,1197.0,2291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11089230,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Pamela Levine,Luxury Partners Realty,Jeff Levine,Luxury Partners Realty,4,blue,Alina Residences at 220 SE Mizner Blvd #215 in...
4,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #217,Boca Raton,FL,33432.0,2900000.0,2.0,2.5,Alina Residences,2429.0,180.0,2024.0,NaN,1194.0,2050.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11089244,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Rochelle LeCavalier,Douglas Elliman,None,None,5,blue,Alina Residences at 220 SE Mizner Blvd #217 in...
5,PAST SALE,2025-05-06,Condo/Co-op,44 Cocoanut Row Unit 321a,Palm Beach,FL,33480.0,2500000.0,2.0,2.0,Palm Beach Towers,1621.0,NaN,1956.0,NaN,1542.0,2193.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Beaches MLS,RX-11071642,N,Y,26.715065,-80.042094,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Mark Whitehead,Coffee Cake And Real Estate,Brie Segota,Coffee Cake And Real Estate,Non Agent,Non-Member Selling Office,None,None,6,blue,Palm Beach Towers at 44 Cocoanut Row Unit 321a...
6,PAST SALE,2025-05-06,Condo/Co-op,604 Captains Way,Jupiter,FL,33477.0,2350000.0,3.0,2.0,Admirals Cove,2160.0,NaN,1988.0,NaN,1088.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/604-Captains...,Beaches MLS,RX-11049155,N,Y,26.909631,-80.085249,https://www.redfin.com/FL/Jupiter/604-Captains...,Stephen Rockoff,The Corcoran Group,None,None,Crystal Condit,Waterfront Properties & Club C,None,None,7,blue,Admirals Cove at 604 Captains Way in Jupiter
7,PAST SALE,2025-05-05,Condo/Co-op,13633 Treasure Cove Cir,North Palm 

In [48]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Two North Breakers Row Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $2,200,000 to $6,000,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, North Palm Beach, Boca Raton, Tequesta, Jupiter, Palm Beach Gardens, Palm Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 97 condo sales totaling $69,559,250 million from May. 4th to May. 11th. The previous week, brokers closed 194 condo sales totaling $96,028,333.

Last week’s units sold for an average of $717,106, higher than 

In [49]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [50]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [51]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [52]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-06,Condo/Co-op,2 N Breakers Row Unit S-T1,Palm Beach,FL,33480.0,6000000.0,3.0,3.5,Two North Breakers Row Condo,3243.0,NaN,1986.0,NaN,1850.0,13078.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10879239,N,Y,26.717140,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Laura Semler,Brown Harris Stevens of PB,None,None,Christian Angle,Christian Angle Real Estate,None,None,1,orange,Two North Breakers Row Condo at 2 N Breakers R...
1,PAST SALE,2025-05-09,Condo/Co-op,425 Beach Rd Ph -O,Tequesta,FL,33469.0,4100000.0,3.0,3.5,Cliveden Jupiter Island,2750.0,NaN,1996.0,NaN,1491.0,2378.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/425-Beach-Rd...,Beaches MLS,RX-11056254,N,Y,26.955362,-80.077420,https://www.redfin.com/FL/Jupiter/425-Beach-Rd...,John Holland,Holland Group Real Estate LLC,Emily Sullivan,Holland Group Real Estate LLC,Traci Chivers,Premier Properties of South Fl,None,None,2,blue,Cliveden Jupiter Island at 425 Beach Rd Ph -O ...
2,PAST SALE,2025-05-07,Condo/Co-op,2700 Donald Ross Rd #412,Palm Beach Gardens,FL,33410.0,3900000.0,4.0,3.5,Azure,3466.0,NaN,2016.0,NaN,1125.0,3635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11049233,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Ken Meierling,Engel & Voelkers Jupiter RE,None,None,3,blue,Azure at 2700 Donald Ross Rd #412 in Palm Beac...
3,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #215,Boca Raton,FL,33432.0,3211000.0,2.0,3.0,Alina Residences,2683.0,180.0,2024.0,NaN,1197.0,2291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11089230,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Pamela Levine,Luxury Partners Realty,Jeff Levine,Luxury Partners Realty,4,blue,Alina Residences at 220 SE Mizner Blvd #215 in...
4,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #217,Boca Raton,FL,33432.0,2900000.0,2.0,2.5,Alina Residences,2429.0,180.0,2024.0,NaN,1194.0,2050.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11089244,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Rochelle LeCavalier,Douglas Elliman,None,None,5,blue,Alina Residences at 220 SE Mizner Blvd #217 in...
5,PAST SALE,2025-05-06,Condo/Co-op,44 Cocoanut Row Unit 321a,Palm Beach,FL,33480.0,2500000.0,2.0,2.0,Palm Beach Towers,1621.0,NaN,1956.0,NaN,1542.0,2193.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Beaches MLS,RX-11071642,N,Y,26.715065,-80.042094,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Mark Whitehead,Coffee Cake And Real Estate,Brie Segota,Coffee Cake And Real Estate,Non Agent,Non-Member Selling Office,None,None,6,blue,Palm Beach Towers at 44 Cocoanut Row Unit 321a...
6,PAST SALE,2025-05-06,Condo/Co-op,604 Captains Way,Jupiter,FL,33477.0,2350000.0,3.0,2.0,Admirals Cove,2160.0,NaN,1988.0,NaN,1088.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/604-Captains...,Beaches MLS,RX-11049155,N,Y,26.909631,-80.085249,https://www.redfin.com/FL/Jupiter/604-Captains...,Stephen Rockoff,The Corcoran Group,None,None,Crystal Condit,Waterfront Properties & Club C,None,None,7,blue,Admirals Cove at 604 Captains Way in Jupiter
7,PAST SALE,2025-05-05,Condo/Co-op,13633 Treasure Cove Cir,North Palm 

In [53]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Two North Breakers Row Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $2,200,000 to $6,000,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, North Palm Beach, Boca Raton, Tequesta, Jupiter, Palm Beach Gardens, Palm Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 97 condo sales totaling $69,559,250 million from May. 4th to May. 11th. The previous week, brokers closed 194 condo sales totaling $96,028,333.

Last week’s units sold for an average of $717,106, higher than 

In [54]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Palm-Beach/2-N-Breakers-Row-33480/unit-S-T1/home/190483993


In [55]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Jupiter/425-Beach-Rd-33469/unit-O/home/42469663


In [56]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/North-Palm-Beach/13629-Treasure-Cove-Cir-33408/home/52844986


In [57]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/North-Palm-Beach/13631-Treasure-Cove-Cir-33408/home/52844984


In [58]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/2-N-Breakers-Row-33480/unit-S-T1/home/190483993


In [59]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/North-Palm-Beach/13633-Treasure-Cove-Cir-33408/home/52844985


In [60]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-06,Condo/Co-op,2 N Breakers Row Unit S-T1,Palm Beach,FL,33480.0,6000000.0,3.0,3.5,Two North Breakers Row Condo,3243.0,NaN,1986.0,NaN,1850.0,13078.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10879239,N,Y,26.717140,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Laura Semler,Brown Harris Stevens of PB,None,None,Christian Angle,Christian Angle Real Estate,None,None,1,orange,Two North Breakers Row Condo at 2 N Breakers R...
1,PAST SALE,2025-05-09,Condo/Co-op,425 Beach Rd Ph -O,Tequesta,FL,33469.0,4100000.0,3.0,3.5,Cliveden Jupiter Island,2750.0,NaN,1996.0,NaN,1491.0,2378.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/425-Beach-Rd...,Beaches MLS,RX-11056254,N,Y,26.955362,-80.077420,https://www.redfin.com/FL/Jupiter/425-Beach-Rd...,John Holland,Holland Group Real Estate LLC,Emily Sullivan,Holland Group Real Estate LLC,Traci Chivers,Premier Properties of South Fl,None,None,2,blue,Cliveden Jupiter Island at 425 Beach Rd Ph -O ...
2,PAST SALE,2025-05-07,Condo/Co-op,2700 Donald Ross Rd #412,Palm Beach Gardens,FL,33410.0,3900000.0,4.0,3.5,Azure,3466.0,NaN,2016.0,NaN,1125.0,3635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11049233,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Ken Meierling,Engel & Voelkers Jupiter RE,None,None,3,blue,Azure at 2700 Donald Ross Rd #412 in Palm Beac...
3,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #215,Boca Raton,FL,33432.0,3211000.0,2.0,3.0,Alina Residences,2683.0,180.0,2024.0,NaN,1197.0,2291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11089230,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Pamela Levine,Luxury Partners Realty,Jeff Levine,Luxury Partners Realty,4,blue,Alina Residences at 220 SE Mizner Blvd #215 in...
4,PAST SALE,2025-05-09,Condo/Co-op,220 SE Mizner Blvd #217,Boca Raton,FL,33432.0,2900000.0,2.0,2.5,Alina Residences,2429.0,180.0,2024.0,NaN,1194.0,2050.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11089244,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Rochelle LeCavalier,Douglas Elliman,None,None,5,blue,Alina Residences at 220 SE Mizner Blvd #217 in...
5,PAST SALE,2025-05-06,Condo/Co-op,44 Cocoanut Row Unit 321a,Palm Beach,FL,33480.0,2500000.0,2.0,2.0,Palm Beach Towers,1621.0,NaN,1956.0,NaN,1542.0,2193.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Beaches MLS,RX-11071642,N,Y,26.715065,-80.042094,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Mark Whitehead,Coffee Cake And Real Estate,Brie Segota,Coffee Cake And Real Estate,Non Agent,Non-Member Selling Office,None,None,6,blue,Palm Beach Towers at 44 Cocoanut Row Unit 321a...
6,PAST SALE,2025-05-06,Condo/Co-op,604 Captains Way,Jupiter,FL,33477.0,2350000.0,3.0,2.0,Admirals Cove,2160.0,NaN,1988.0,NaN,1088.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/604-Captains...,Beaches MLS,RX-11049155,N,Y,26.909631,-80.085249,https://www.redfin.com/FL/Jupiter/604-Captains...,Stephen Rockoff,The Corcoran Group,None,None,Crystal Condit,Waterfront Properties & Club C,None,None,7,blue,Admirals Cove at 604 Captains Way in Jupiter
7,PAST SALE,2025-05-05,Condo/Co-op,13633 Treasure Cove Cir,North Palm 

## Time on Market Calculator

In [63]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 1, 27) ## List (Earlier) date
date2 = datetime(2025, 5, 9) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

102


## Clean CSV for Datawrapper Chart

In [64]:
chart_df = df_top_ten

In [65]:
chart_df = chart_df.fillna(" ")

In [66]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [67]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [68]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [69]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [70]:
chart_df['$/SQUARE FEET'].astype(float)

0    1850.0
1    1491.0
2    1125.0
3    1197.0
4    1194.0
5    1542.0
6    1088.0
7     882.0
8     929.0
9    1146.0
Name: $/SQUARE FEET, dtype: float64

In [71]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [72]:
csv_date_string = today.strftime("%m_%d_%Y")

In [73]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [74]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-05-06,2 N Breakers Row Unit S-T1 in Palm Beach,"$6,000,000",3,3.5,"3,243",1986,"$1,850",Beaches MLS,RX-10879239,Laura Semler Brown Harris Stevens of PB,Christian Angle Christian Angle Real Estate
1,2025-05-09,425 Beach Rd Ph -O in Tequesta,"$4,100,000",3,3.5,"2,750",1996,"$1,491",Beaches MLS,RX-11056254,John Holland Holland Group Real Estate LLC Emi...,Traci Chivers Premier Properties of South Fl
2,2025-05-07,2700 Donald Ross Rd #412 in Palm Beach Gardens,"$3,900,000",4,3.5,"3,466",2016,"$1,125",Beaches MLS,RX-11049233,Bradley Ball Frankel Ball Realty LLC Shannon B...,Ken Meierling Engel & Voelkers Jupiter RE
3,2025-05-09,220 SE Mizner Blvd #215 in Boca Raton,"$3,211,000",2,3.0,"2,683",2024,"$1,197",Beaches MLS,RX-11089230,Rochelle LeCavalier Douglas Elliman Kathy Koch...,Pamela Levine Luxury Partners Realty Jeff Levi...
4,2025-05-09,220 SE Mizner Blvd #217 in Boca Raton,"$2,900,000",2,2.5,"2,429",2024,"$1,194",Beaches MLS,RX-11089244,Rochelle LeCavalier Douglas Elliman Kathy Koch...,Rochelle LeCavalier Douglas Elliman
5,2025-05-06,44 Cocoanut Row Unit 321a in Palm Beach,"$2,500,000",2,2.0,"1,621",1956,"$1,542",Beaches MLS,RX-11071642,Mark Whitehead Coffee Cake And Real Estate Bri...,Non Agent Non-Member Selling Office
6,2025-05-06,604 Captains Way in Jupiter,"$2,350,000",3,2.0,"2,160",1988,"$1,088",Beaches MLS,RX-11049155,Stephen Rockoff The Corcoran Group,Crystal Condit Waterfront Properties & Club C
7,2025-05-05,13633 Treasure Cove Cir in North Palm Beach,"$2,200,000",3,2.0,"2,493",2012,$882,Beaches MLS,RX-11048862,"James Edwards Sotheby's Intl. Realty, Inc. Pat...",Patrick Kelly Coastal Realty Group Sales & M
8,2025-05-05,13631 Treasure Cove Cir in North Palm Beach,"$2,200,000",3,2.0,"2,367",2012,$929,Beaches MLS,RX-11048869,"James Edwards Sotheby's Intl. Realty, Inc. Pat...",Patrick Kelly Coastal Realty Group Sales & M
9,2025-05-05,13629 Treasure Cove Cir in North Palm Beach,"$2,200,000",3,2.0,"1,920",2012,"$1,146",Beaches MLS,RX-11048850,"James Edwards Sotheby's Intl. Realty, Inc. Pat...",Patrick Kelly Coastal Realty Group Sales & M
